Metrics on pre and post finetuning models



In [1]:
import torch
import torch.nn.functional as F
from transformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
device = "cuda" if torch.cuda.is_available() else "cpu"

In [1]:
def load_model_and_tokenizer(model_name):
    print(f"Using device: {device}\n")
    print(f"Loading model: {model_name}")
    model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    return model, tokenizer

In [1]:
# TODO: change these (can be huggingface repo names or paths to local models)
MODEL_A = '/mnt/data/llms/Qwen2.5-1.5B-Instruct-finetuned/checkpoint-10-of-10/'
MODEL_B = '/mnt/data/llms/Qwen2.5-1.5B-Instruct/'

# NOTE: tokenizers for the models should be the same
model_a, tokenizer = load_model_and_tokenizer(MODEL_A)
model_b, _ = load_model_and_tokenizer(MODEL_B)

In [1]:
def get_stats_for_generation(model, tokenizer, generated_ids, prompt_length):
    device = model.device
    inputs = {"input_ids": generated_ids.to(device)}

    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=False, output_attentions=False)
        logits = outputs.logits

    stats = []
    for j in range(prompt_length - 1, generated_ids.shape[1] - 1):
        pos_logits = logits[0, j, :]
        entropy = torch.distributions.Categorical(logits=pos_logits).entropy().item()
        actual_next_token_id = generated_ids[0, j + 1]
        probabilities = F.softmax(pos_logits, dim=-1)
        prob_of_chosen_token = probabilities[actual_next_token_id].item()
        current_token_str = tokenizer.decode(generated_ids[0, j])
        next_token_str = tokenizer.decode(actual_next_token_id)
        stats.append({
            'current_token': current_token_str,
            'next_token': next_token_str,
            'entropy': entropy,
            'probability_of_next_token': prob_of_chosen_token,
        }) # had used token-wise KL divergence as well for ground truth experiment as well
        
    return pd.DataFrame(stats)

def process_generations(model_a, model_b, tokenizer, prompts: list[str], max_new_tokens: int = 256):
    device_a = model_a.device
    device_b = model_b.device
    all_outputs = []

    for i, prompt in enumerate(prompts):
        print(f"\n\n{'=' * 35} PROMPT {i + 1}/{len(prompts)} {'=' * 35}")
        formatted_prompt = tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)
        inputs = tokenizer(formatted_prompt, return_tensors="pt")
        prompt_length = inputs.input_ids.shape[1]
        print(f"PROMPT: \"{formatted_prompt[:500]}...\"\n")
        print("-" * 30)
        print("Generating with Model A...")
        generated_ids_a = model_a.generate(
            inputs.input_ids.to(device_a),
            attention_mask=inputs.attention_mask.to(device_a),
            max_new_tokens=max_new_tokens,
            pad_token_id=tokenizer.eos_token_id
        ) # sampling parameters from generation_config.json in the model checkpoint
        stats_a = get_stats_for_generation(model_a, tokenizer, generated_ids_a, prompt_length)
        full_text_a = tokenizer.decode(generated_ids_a[0], skip_special_tokens=True)
        print("Done.\n")
        print("Generating with Model B...")
        generated_ids_b = model_b.generate(
            inputs.input_ids.to(device_b),
            attention_mask=inputs.attention_mask.to(device_b),
            max_new_tokens=max_new_tokens,
            pad_token_id=tokenizer.eos_token_id
        ) # sampling parameters from generation_config.json in the model checkpoint
        stats_b = get_stats_for_generation(model_b, tokenizer, generated_ids_b, prompt_length)
        full_text_b = tokenizer.decode(generated_ids_b[0], skip_special_tokens=True)
        print("Done.")
        print("-" * 30)
        all_outputs.append({
            "prompt": formatted_prompt,
            "model_a_stats": stats_a,
            "model_b_stats": stats_b,
            "model_a_text": full_text_a,
            "model_b_text": full_text_b,
        })
    return all_outputs

def process_teacher_forced(model_a, model_b, tokenizer, prompts: list[str]):
    device_a = model_a.device
    device_b = model_b.device
    all_outputs = []
    for i, prompt in enumerate(prompts):
        print(f"\n\n{'=' * 35} PROMPT {i + 1}/{len(prompts)} {'=' * 35}")
        formatted_prompt = tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=False)
        inputs = tokenizer(formatted_prompt, return_tensors="pt")
        prompt_length = inputs.input_ids.shape[1]
        print(f"PROMPT: \"{formatted_prompt[:500]}...\"\n")
        print("-" * 30)
        print("Generating with Model A...")
        generated_ids_a = inputs.input_ids.to(device_a)
        stats_a = get_stats_for_generation(model_a, tokenizer, generated_ids_a, 1)
        full_text_a = tokenizer.decode(generated_ids_a[0], skip_special_tokens=True)
        print("Done.\n")
        print("Generating with Model B...")
        generated_ids_b = inputs.input_ids.to(device_b),
        stats_b = get_stats_for_generation(model_b, tokenizer, generated_ids_b, 1)
        full_text_b = tokenizer.decode(generated_ids_b[0], skip_special_tokens=True)
        print("Done.")
        print("-" * 30)
        all_outputs.append({
            "prompt": formatted_prompt,
            "model_a_stats": stats_a,
            "model_b_stats": stats_b,
            "model_a_text": full_text_a,
            "model_b_text": full_text_b,
        })
    return all_outputs

Prompt-by-prompt analysis:

Note: the following are some prompts sampled completely randomly from an eval set containing of an OOM more samples (was in a jsonl, sampled here for reproducibility). For logits on ground truths, you'd need to add assistant messages here too, and use \`process<sub>teacher</sub><sub>forced</sub>\` instead of \`process<sub>generations</sub>\`.



In [1]:
PROMPTS = [
    [{"role": "system", "content": "You are a helpful assistant. Think step by step before responding to the user's query. Your thought process should be enclosed between <think> and </think> tags. Once your thought process is complete, write a response which should end in the final answer enclosed in \\boxed{}."}, {"role": "user", "content": "An unbiased coin is flipped repeatedly to generate a sequence of H's (heads) and T's (tails). The probability of getting exactly two H's in the first three flips is 3/8. If we ignore the first three flips and only consider the next four flips, what is the probability that exactly two of the next four flips are H's?"}, ],
    [{"role": "system", "content": "You are a helpful assistant. Think step by step before responding to the user's query. Your thought process should be enclosed between <think> and </think> tags. Once your thought process is complete, write a response which should end in the final answer enclosed in \\boxed{}."}, {"role": "user", "content": "Given the parametric equations \\(x = 3t^2 + 1\\) and \\(y = 2t - 1\\), eliminate the parameter \\(t\\) and find the equation of the curve."}, ],
    [{"role": "system", "content": "You are a helpful assistant. Think step by step before responding to the user's query. Your thought process should be enclosed between <think> and </think> tags. Once your thought process is complete, write a response which should end in the final answer enclosed in \\boxed{}."}, {"role": "user", "content": "The temperature at a location is 95 degrees Fahrenheit and rising at a rate of 2 degrees Fahrenheit per hour. At the same time, a thermometer is being used at a different location 3 miles away. The initial temperature reading at this location is 70 degrees Fahrenheit and rising at a rate of 4 degrees Fahrenheit per hour. How many hours will it take for the temperature at the two locations to be the same?"}, ],
    [{"role": "system", "content": "You are a helpful assistant. Think step by step before responding to the user's query. Your thought process should be enclosed between <think> and </think> tags. Once your thought process is complete, write a response which should end in the final answer enclosed in \\boxed{}."}, {"content": "Consider the terms of an arithmetic sequence: $-\\frac{1}{3}, y+2, 4y, \\ldots$. Solve for $y$.", "role": "user"}, ],
    [{'role': 'system', 'content': "You are a helpful assistant. Think step by step before responding to the user's query. Your thought process should be enclosed between <think> and </think> tags. Once your thought process is complete, write a response which should end in the final answer enclosed in \\boxed{}."}, {'role': 'user', 'content': 'You have 10 balls and you want to put them into 3 boxes. How many ways can you distribute the balls into the boxes?\nThe boxes are distinguishable but the balls are indistinguishable.\nThis is a classic stars and bars problem.\nThe formula for this problem is n+k-1 choose k-1, where n is the number of balls and k is the number of boxes. \nApply the formula and find the result.'}],
    [{'role': 'system', 'content': "You are a helpful assistant. Think step by step before responding to the user's query. Your thought process should be enclosed between <think> and </think> tags. Once your thought process is complete, write a response which should end in the final answer enclosed in \\boxed{}."}, {'role': 'user', 'content': "One possible mechanism for the formation of the fullerenes $C_{60}$ from graphite is given by the following steps:\n\nStep 1: $C_2$ fragments are split from graphite, which gives a set amount of energy per $C_2$ unit (we will call it $\\Delta{E}_1$).\nStep 2: The $C_2$ fragments then combine to form rings, with each ring having a certain number of carbon atoms. There is an associated energy change with this step (we will call it $\\Delta{E}_2$).\nStep 3: Finally, the rings condense into fullerenes with energy change $\\Delta{E}_3$. \nFor our purposes, let's just consider the number of $C_2$ fragments."}],
    [{'role': 'system', 'content': "You are a helpful assistant. Think step by step before responding to the user's query. Your thought process should be enclosed between <think> and </think> tags. Once your thought process is complete, write a response which should end in the final answer enclosed in \\boxed{}."}, {'role': 'user', 'content': 'A plane flies at an average speed of 500 mph, and it travels 2500 miles in 5 hours. How much more fuel will it need to travel 600 miles more at the same speed?'}],
    [{'role': 'system', 'content': "You are a helpful assistant. Think step by step before responding to the user's query. Your thought process should be enclosed between <think> and </think> tags. Once your thought process is complete, write a response which should end in the final answer enclosed in \\boxed{}."}, {'role': 'user', 'content': 'The problem states: Consider the equation of the circle with center $(h, k)$ and a point on the circle at $(x_1,y_1)$. Write the equation of the circle in the standard form $(x - h)^2 + (y - k)^2 = r^2$, where $r$ is the radius of the circle.'}],
    [{'role': 'system', 'content': "You are a helpful assistant. Think step by step before responding to the user's query. Your thought process should be enclosed between <think> and </think> tags. Once your thought process is complete, write a response which should end in the final answer enclosed in \\boxed{}."}, {'role': 'user', 'content': '# Problem\nGiven the following quotient and remainder obtained from dividing $n$ by $m$:\n\n$$\\frac{n}{m} = 5 + \\frac{2}{m}$$\n\nWhat is the value of $n$ in terms of $m$?'}],
    [{'role': 'system', 'content': "You are a helpful assistant. Think step by step before responding to the user's query. Your thought process should be enclosed between <think> and </think> tags. Once your thought process is complete, write a response which should end in the final answer enclosed in \\boxed{}."}, {'role': 'user', 'content': 'Consider the function \\(f(x) = x^2\\) on the interval \\([0, 2]\\). We want to find the area under the curve of \\(f(x)\\) from \\(x = 0\\) to \\(x = 2\\).'}],
    [{'role': 'system', 'content': "You are a helpful assistant. Think step by step before responding to the user's query. Your thought process should be enclosed between <think> and </think> tags. Once your thought process is complete, write a response which should end in the final answer enclosed in \\boxed{}."}, {'role': 'user', 'content': 'A survey of 250 families in a village was conducted to determine the average number of cars per family. The average number of cars per family for the entire village was 1.22. If the average number of cars per family for the richest 10% of the families was 2.5, what is the average number of cars per family for the remaining 90% of the families?'}],
    [{'role': 'system', 'content': "You are a helpful assistant. Think step by step before responding to the user's query. Your thought process should be enclosed between <think> and </think> tags. Once your thought process is complete, write a response which should end in the final answer enclosed in \\boxed{}."}, {'role': 'user', 'content': 'Yesterday, a certain country had 10 new cases of a disease. Today, there were 15 new cases of the disease. If the disease is spreading at a steady rate, find the number of new cases tomorrow.'}],
    [{'role': 'system', 'content': "You are a helpful assistant. Think step by step before responding to the user's query. Your thought process should be enclosed between <think> and </think> tags. Once your thought process is complete, write a response which should end in the final answer enclosed in \\boxed{}."}, {'role': 'user', 'content': 'Based on the tracker data, the object was moving at a speed of $100$ km/h when it crossed a speed detector. Later, it was observed that the speed began to decrease, causing it to slow down to $50$ km/h over a distance of $50$ km. Determine a suitable model for the speed of the object as a function of distance traveled.'}],
    [{'role': 'system', 'content': "You are a helpful assistant. Think step by step before responding to the user's query. Your thought process should be enclosed between <think> and </think> tags. Once your thought process is complete, write a response which should end in the final answer enclosed in \\boxed{}."}, {'role': 'user', 'content': 'Consider the equation \\[\\frac{2}{2pt+1} = \\frac{1}{pt+3},\\] where $p$ and $t$ are constants. Solve for $t$.'}],
    [{'role': 'system', 'content': "You are a helpful assistant. Think step by step before responding to the user's query. Your thought process should be enclosed between <think> and </think> tags. Once your thought process is complete, write a response which should end in the final answer enclosed in \\boxed{}."}, {'role': 'user', 'content': 'What is the value of $x$ in the equation $x^2 + 2x - 6 = 0$?'}],
    [{'role': 'system', 'content': "You are a helpful assistant. Think step by step before responding to the user's query. Your thought process should be enclosed between <think> and </think> tags. Once your thought process is complete, write a response which should end in the final answer enclosed in \\boxed{}."}, {'role': 'user', 'content': "How do you calculate the truth table of an implication in propositional logic? \n\nTo calculate the truth table of the implication $p \\implies q$, the following rules are used: \n\n* If $p$ is false, then $p \\implies q$ is true.\n* If $p$ is true and $q$ is true, then $p \\implies q$ is true.\n* If $p$ is true and $q$ is false, then $p \\implies q$ is false.\n\nUsing these rules, calculate the truth table of $p \\implies q$. \n\nPlease provide the full truth table in your final answer. \n\nNote: Use 'T' to denote true and 'F' to denote false."}],
    [{'role': 'system', 'content': "You are a helpful assistant. Think step by step before responding to the user's query. Your thought process should be enclosed between <think> and </think> tags. Once your thought process is complete, write a response which should end in the final answer enclosed in \\boxed{}."}, {'role': 'user', 'content': "Based on historical data, you know that 60% of male Romanticsists are liberal and 70% of female Romantics are conservative. Assuming that the base rate of Romanticists is 10% of the population, and that the population is evenly split between males and females, you receive a tip that someone is a Romanticist. What is the probability that the tip is referring to a male given that the person is a liberal, and how can you apply Bayes' theorem to calculate this probability?"}],
    [{'role': 'system', 'content': "You are a helpful assistant. Think step by step before responding to the user's query. Your thought process should be enclosed between <think> and </think> tags. Once your thought process is complete, write a response which should end in the final answer enclosed in \\boxed{}."}, {'role': 'user', 'content': 'Let $a_n$ be defined in the following way:\n$a_0 = 1$\n$a_1 = 3$\nFor $n \\ge 2$, $a_n = 5 a_{n-1} - 3a_{n-2}$. Thus, $a_2 = 5a_1-3a_0 = 12$, and $a_3=5a_2-3a_1=45$.\nStarting from $n=0$, list the first 10 values of the sequence $a_n$.'}],
    [{'role': 'system', 'content': "You are a helpful assistant. Think step by step before responding to the user's query. Your thought process should be enclosed between <think> and </think> tags. Once your thought process is complete, write a response which should end in the final answer enclosed in \\boxed{}."}, {'role': 'user', 'content': "A bakery is famous for its chocolate cake. It makes only one size of cake, which weighs 680 grams. The bakery's recipe for the cake requires 250 grams of chocolate. If the bakery starts with 20 kilograms of chocolate and it uses up all of the chocolate in making the cakes, how many cakes can it make?"}],
    [{'role': 'system', 'content': "You are a helpful assistant. Think step by step before responding to the user's query. Your thought process should be enclosed between <think> and </think> tags. Once your thought process is complete, write a response which should end in the final answer enclosed in \\boxed{}."}, {'role': 'user', 'content': 'A poll shows that half of all Internet users have visited a chat room at least once, 1/3 have visited an online shopping site, and 1/5 have visited an online news or magazine site. What percentage of users have visited either a chat room, an online shopping site, or an online news site?\nIf you want the answer in percentage form, multiply the result of the calculation by 100.'}],
    [{'role': 'system', 'content': "You are a helpful assistant. Think step by step before responding to the user's query. Your thought process should be enclosed between <think> and </think> tags. Once your thought process is complete, write a response which should end in the final answer enclosed in \\boxed{}."}, {'role': 'user', 'content': 'What is the value of x in the equation 2e^(2x) + 3e^(x) - 2 = 0?'}],
    [{'role': 'system', 'content': "You are a helpful assistant. Think step by step before responding to the user's query. Your thought process should be enclosed between <think> and </think> tags. Once your thought process is complete, write a response which should end in the final answer enclosed in \\boxed{}."}, {'role': 'user', 'content': 'The first four terms of a geometric series are:  \n12, 6, 3, 3/2, ...  \nThe problem asks to write the formula for the nth term of the series, and to determine the sum of the infinite series.'}],
    [{'role': 'system', 'content': "You are a helpful assistant. Think step by step before responding to the user's query. Your thought process should be enclosed between <think> and </think> tags. Once your thought process is complete, write a response which should end in the final answer enclosed in \\boxed{}."}, {'role': 'user', 'content': 'In a group of 10 friends, everyone shakes hands with everyone else. How many handshakes take place?'}],
    [{'role': 'system', 'content': "You are a helpful assistant. Think step by step before responding to the user's query. Your thought process should be enclosed between <think> and </think> tags. Once your thought process is complete, write a response which should end in the final answer enclosed in \\boxed{}."}, {'role': 'user', 'content': 'Can you calculate the probability of getting a sum of 12 when rolling 3 dice?\n\nTo calculate the probability, use the following steps:\n\n1. Calculate the total number of possible outcomes when rolling 3 dice.\n2. Count the number of outcomes that give a sum of 12.'}],
]

all_results = process_generations(model_a, model_b, tokenizer, PROMPTS, max_new_tokens=4096)

for INDEX in range(len(PROMPTS)):
    result = all_results[INDEX]
    
    df_a = result['model_a_stats']
    df_b = result['model_b_stats']
    
    print("\n\n" + "=" * 30 + " ANALYSIS " + "=" * 30)
    print(f"Prompt {INDEX + 1}\n")
    
    print("--- Model A (More finetuned) ---")
    print(result['model_a_text'])
    print("\n" + "-" * 40 + "\n")
    
    print("--- Model B (Less finetuned) ---")
    print(result['model_b_text'])
    print("\n" + "=" * 70 + "\n")
    
    print("Entropy statistics for Model A's generation:")
    print(df_a['entropy'].describe())
    print("\n" + "-" * 40 + "\n")
    
    print("Entropy statistics for Model B's generation:")
    print(df_b['entropy'].describe())
    print("\n" + "=" * 70 + "\n")
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(20, 12), sharex=True)
    fig.suptitle(f"Entropy per generation step for prompt {INDEX+1}", fontsize=16)
    
    df_a['entropy'].plot(ax=ax1, style='-o', legend=True, label='Entropy for model A')
    ax1.set_ylabel('Entropy')
    ax1.set_title('Model A generation')
    ax1.grid(True, linestyle='--', alpha=0.6)
    
    df_b['entropy'].plot(ax=ax2, style='-o', legend=True, color='orange', label='Entropy for model B')
    ax2.set_xlabel('Generation step (token index)')
    ax2.set_ylabel('Entropy')
    ax2.set_title('Model B generation')
    ax2.grid(True, linestyle='--', alpha=0.6)
    
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()
    
    fig, ax = plt.subplots(figsize=(12, 7))
    df_a['entropy'].hist(ax=ax, bins=30, alpha=0.7, label='Model A entropy', density=True, log=True)
    df_b['entropy'].hist(ax=ax, bins=30, alpha=0.7, label='Model B entropy', density=True, log=True)
    ax.set_title('Distribution of entropy during generation')
    ax.set_xlabel('Entropy')
    ax.set_ylabel('Density')
    ax.legend()
    plt.show()

In [1]:
def analyze_token_qcuts(df: pd.DataFrame, metric_column: str, model_name: str, n_qcuts: int = 10):
    print("\n" + "=" * 30)
    print(f"Token qcuts for {model_name} based on '{metric_column}'")
    print("=" * 30)
    
    qcut_col_name = f'{metric_column}_qcut'
    df[qcut_col_name] = pd.qcut(df[metric_column], n_qcuts, labels=False, duplicates='drop')
    qcut_intervals = pd.qcut(df[metric_column], n_qcuts, duplicates='drop')
    interval_map = {i: interval for i, interval in enumerate(qcut_intervals.cat.categories)}
    
    num_actual_qcuts = df[qcut_col_name].nunique()
    qcuts_to_display = sorted(list(set([0, num_actual_qcuts // 4, num_actual_qcuts // 2, num_actual_qcuts * 3 // 4, num_actual_qcuts - 1])))

    for qcut_idx in qcuts_to_display:
        df_qcut = df[df[qcut_col_name] == qcut_idx].copy()
        df_qcut.sort_values(by=metric_column, inplace=True)
        print(f"\n--- qcut {qcut_idx + 1}/{num_actual_qcuts} (range: {interval_map[qcut_idx]}) ---")
        num_samples = min(5, len(df_qcut))
        step = max(1, len(df_qcut) // num_samples)
        
        print(df_qcut[['current_token', 'next_token', metric_column]].iloc[::step])

Aggregate stats across prompts:



In [1]:
print(f"\n\n{'#' * 25} Aggregate across all prompts {'#' * 25}")

all_stats_a = [result['model_a_stats'] for result in all_results]
agg_df_a = pd.concat(all_stats_a, ignore_index=True)

all_stats_b = [result['model_b_stats'] for result in all_results]
agg_df_b = pd.concat(all_stats_b, ignore_index=True)

print(f"\nAggregated a total of {len(agg_df_a)} tokens for Model A.")
print(f"Aggregated a total of {len(agg_df_b)} tokens for Model B.")

analyze_token_qcuts(agg_df_a, 'entropy', 'Model A (SFT) - aggregate', n_qcuts=10)
analyze_token_qcuts(agg_df_b, 'entropy', 'Model B (Base) - aggregate', n_qcuts=10)

fig, ax = plt.subplots(figsize=(12, 7))

agg_df_a['entropy'].hist(ax=ax, bins=50, alpha=0.7, label='Model A - all prompts', density=True, log=True)
agg_df_b['entropy'].hist(ax=ax, bins=50, alpha=0.7, label='Model B - all prompts', density=True, log=True)

ax.set_title('Aggregated distribution of token entropies across all generations')
ax.set_xlabel('Entropy')
ax.set_ylabel('Density')
ax.legend()
plt.show()

Looking at high/low entropy tokens:



In [1]:
print(agg_df_a.describe().to_string())
print(agg_df_a.columns.to_list())
agg_df_a_mean = agg_df_a.groupby('current_token')[['entropy', 'probability_of_next_token']].mean()
print(agg_df_a_mean.describe().to_string())
print(agg_df_a_mean.columns.to_list())
print(agg_df_a_mean.sort_values(by='entropy').iloc[-100:].sample(10, random_state=None).to_string())
print(agg_df_a_mean.sort_values(by='entropy').iloc[:100].sample(10, random_state=None).to_string())
agg_df_a_mean = agg_df_a.groupby('next_token')[['entropy', 'probability_of_next_token']].mean()
print(agg_df_a_mean.describe().to_string())
print(agg_df_a_mean.columns.to_list())
print(agg_df_a_mean.sort_values(by='entropy').iloc[-100:].sample(10, random_state=None).to_string())
print(agg_df_a_mean.sort_values(by='entropy').iloc[:100].sample(10, random_state=None).to_string())

Observations on a preliminary run:

1.  For their own generations (no teacher-forcing), using the same sampling parameters
    -   Between the instruct model and the last checkpoint of the finetune, it seems that the finetune has a higher token entropy on average, but the instruct model is heavier tailed. Note that this is done vaguely, need to do it properly (more datapoints). During training, entropy seems to first increase and then decrease (preliminarily checked on the first and last checkpoints and instruct model). Could be due to SFT data being OOD/the model just hedging against high KL-div by assigning non-zero probabilities to improbable tokens as well.
    -   (Random idea) Tail stuff might lead to over-indexing on the high entropy token lottery and worse performance (think pass@k)
    -   Does tail entropy/entropy decrease have a relation with [https://arxiv.org/pdf/2505.22617](https://arxiv.org/pdf/2505.22617)?
    -   The branching interpretation of entropy holds - the lowest next-token-distribution entropy is for tokens that are parts of words/punctuation/math, and the highest is for open-ended English words. Might be related to aha tokens, didn't see something super clear. Might need more experimentation with other models as well as better metrics.

2.  For teacher forced generations (ground truth of different eval sets):
    -   The instruct model generally has somewhat lower entropy on older-CoT-like responses - no self-corrections, no explicitly anthropomorphic reasoning processes. The reasoning SFT model has lower entropy on modern CoT-like responses too. I think this might be due to the training data distributions, though could be worth looking into.
    -   The branching interpretation of entropy holds here too.

TODOs:

1.  Multi-checkpoint analysis.
2.  Better statistical significance for experiments.
3.  Designing better metrics to identify specific important tokens.
4.  Run this for other series of base models and finetuned checkpoints (both on task-specific and task-agnostic finetunes).

